In [ ]:
!eups list -s | grep lsst_distrib

In [ ]:
import numpy as np
import astropy.units as u
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from scipy.spatial import KDTree
import scipy.interpolate as interpolate
from scipy.optimize import curve_fit
import scipy.stats as stats
import healpy as hp
import pandas as pd
from matplotlib import cm
from astropy.visualization import make_lupton_rgb
from astropy.io import fits
# %matplotlib widget

In [ ]:
# Familiar stack packages
from lsst.daf.butler import Butler
from lsst.geom import Box2I, Box2D, Point2I, Point2D, Extent2I, Extent2D
from lsst.afw.image import Exposure, Image, PARENT
import lsst.sphgeom

# These may be less familiar objects dealing with multi-band data products
from lsst.afw.image import MultibandExposure, MultibandImage
from lsst.afw.detection import MultibandFootprint
from lsst.afw.image import MultibandExposure

In [ ]:
from lsst.meas.algorithms import SourceDetectionTask
from lsst.meas.algorithms import DynamicDetectionTask
from lsst.meas.extensions.scarlet import ScarletDeblendTask
from lsst.meas.base import SingleFrameMeasurementTask
from lsst.afw.table import SourceCatalog

import lsst.scarlet.lite as scl
# import scarlet

In [ ]:
from lsst.daf.butler import Butler

In [ ]:
def showRGB(image, bgr="gri", ax=None, fp=None, figsize=(8,8), stretch=1, Q=10):
    """Display an RGB color composite image with matplotlib.
    
    Parameters
    ----------
    image : `MultibandImage`
        `MultibandImage` to display.
    bgr : sequence
        A 3-element sequence of filter names (i.e. keys of the exps dict) indicating what band
        to use for each channel. If `image` only has three filters then this parameter is ignored
        and the filters in the image are used.
    ax : `matplotlib.axes.Axes`
        Axis in a `matplotlib.Figure` to display the image.
        If `axis` is `None` then a new figure is created.
    fp: `lsst.afw.detection.Footprint`
        Footprint that contains the peak catalog for peaks in the image.
        If `fp` is `None` then no peak positions are plotted.
    figsize: tuple
        Size of the `matplotlib.Figure` created.
        If `ax` is not `None` then this parameter is ignored.
    stretch: int
        The linear stretch of the image.
    Q: int
        The Asinh softening parameter.
    """
    # If the image only has 3 bands, reverse the order of the bands to produce the RGB image
    if len(image) == 3:
        bgr = image.filters
    # Extract the primary image component of each Exposure with the .image property, and use .array to get a NumPy array view.
    rgb = make_lupton_rgb(image_r=image[bgr[2]].array,  # numpy array for the r channel
                          image_g=image[bgr[1]].array,  # numpy array for the g channel
                          image_b=image[bgr[0]].array,  # numpy array for the b channel
                          stretch=stretch, Q=Q)  # parameters used to stretch and scale the pixel values
    if ax is None:
        fig = plt.figure(figsize=figsize)
        ax = fig.add_subplot(1,1,1)
    
    # Exposure.getBBox() returns a Box2I, a box with integer pixel coordinates that correspond to the centers of pixels.
    # Matplotlib's `extent` argument expects to receive the coordinates of the edges of pixels, which is what
    # this Box2D (a box with floating-point coordinates) represents.
    integerPixelBBox = image[bgr[0]].getBBox()
    bbox = Box2D(integerPixelBBox)
    ax.imshow(rgb, interpolation='nearest', origin='lower', extent=(bbox.getMinX(), bbox.getMaxX(), bbox.getMinY(), bbox.getMaxY()))
    if fp is not None:
        for peak in fp.getPeaks():
            ax.plot(peak.getIx(), peak.getIy(), "bx", mew=2)

In [ ]:
arcsec = 1/60**2

In [ ]:
def match_unrec_blends(obs_coords, space_mags, obs_tree, space_tree,
                       min_mag=28.5, max_diff=2, obj_radius=1*arcsec, space_radius=1*arcsec):
    '''
    Matching function to label unrecognized blends. 
    '''
    obj_blend_row = np.zeros(len(obs_coords))
    magdiff_unrec_blends = []
    for ondx,oc in enumerate(obs_coords):
        truths = space_tree.query_ball_point(oc, r=space_radius)
        objs = obs_tree.query_ball_point(oc, r=obj_radius)
    
        Nt = len(truths) 
        No = len(objs)

        if (Nt - No) > 0:
            match_mags = np.zeros(len(truths))
            for i,t_ndx in enumerate(truths):
                match_mags[i] = space_mags[t_ndx]

            diff_mags = match_mags - match_mags.min()
            diff_filter = (diff_mags<=max_diff) * (match_mags < min_mag)
            filtered_mags = np.array(match_mags)[diff_filter]
            
            if (len(filtered_mags) - len(objs))>=1:
                filt_ndx = np.array(truths)[diff_filter]
                magdiff_unrec_blends.append([filtered_mags,filt_ndx, ondx])
                for o_ndx in objs:
                    obj_blend_row[o_ndx] = 1
    return obj_blend_row.astype(bool), magdiff_unrec_blends
                

In [ ]:
def fblend_generic_1d(gen_row, blend_row, add_filt=None, b_bins=None, Nbin=50):
    '''
    Helper function to calculate fraction of unrecognized blend as a function of `gen_row` binning.

    `gen_row` : Generic row. Any array of the same length as `blend_row` that has a variable we want to bin on
    `blend_row` : Blend label row. Array of booleans labelling an object as pure (0) or blended (1)
    `add_filt` : Additional filter. Any additional filter that should be applied along with binning. Default is no additional filter
    `b_bins` :  Blend bins. User supplied binning of `gen_row.` If no bins are provided, automatically create from minimum to maximum of `gen_row` with `Nbin` bins
    `Nbin` : Number of bins. When using automatic binning, the number of bins to use. 
    '''
    # gen_row is any measurement with the same size of blend_row

    if b_bins is None:
        b_bins = np.linspace(gen_row.min(), gen_row.max(), Nbin)
    else:
        Nbin = len(b_bins)
    
    f_blend_mag = np.zeros(Nbin)
    f_blend_err = np.zeros_like(f_blend_mag)
    f_bins = np.zeros_like(f_blend_mag)
    
    # b_bins = np.linspace(14.5, 30, 50)
    f_bins = b_bins
    b_digitize = np.digitize(gen_row, b_bins)

    num_bin = len(b_bins)
    
    for ndx in range(1, num_bin+1):
        if add_filt is None:
            filt = (b_digitize==ndx)
        else:
            filt = (b_digitize==ndx) * add_filt
        if (np.sum(filt)==0):
            continue
        # num_blends = np.sum(obj_tract[filt]['unrec_blend_score'] > unrec_cutoff)
        num_blends = np.sum(blend_row[filt])
        num_objs = np.sum(filt)

        f_blend_mag[ndx-1] = num_blends/num_objs
        # Treating num_blends and num_objs as independent poisson
        f_blend_err[ndx-1] = (num_blends + num_blends ** 2 * num_objs / num_objs ** 2) ** (1 / 2) / num_objs
    
    return {'fraction': f_blend_mag, 'err': f_blend_err, 'bins': f_bins}

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format='retina'

In [ ]:
arcsec = 1/60**2
obs_repo = '/repo/embargo'

In [ ]:
# Relevant bands
# bands = list('ugrizy')
bands = list('gri')
blen = len(bands)

# Column names for the bands

ap_suffix = '_ap12Flux'
kron_suffix = '_kronFlux'

aper_bands = [b+ap_suffix for b in bands]
kron_bands = [b+kron_suffix for b in bands]

### HST Data

In [ ]:
catalog_dir = '/sdf/data/rubin/user/dtaranu/tickets/cdfs/'
catalog_fname = f"{catalog_dir}/hlsp_hlf_hst_60mas_goodss_v2.1_catalog.fits"

In [ ]:
hst_fits = fits.open(catalog_fname)

In [ ]:
hst_ra = hst_fits[1].data['ra']
hst_dec = hst_fits[1].data['dec']

hst_data = np.vstack((hst_ra, hst_dec)).T
hst_tree = KDTree(hst_data)

In [ ]:
hst_f814_all = hst_fits[1].data['f_f814w']

In [ ]:
# Need to double check the right way to get the magnitudes.
# Using AB zeropoint from: https://acstools.readthedocs.io/en/latest/acszpt.html

hst_f814_mags = -2.5*np.log10(hst_f814_all, where=hst_f814_all>-99, out=-99*np.ones(len(hst_f814_all))) + 25.937# + 25.69 + 25.937

In [ ]:
print("HST Data bounds: ", hst_data[:,0].min(), hst_data[:,0].max(), hst_data[:,1].min(), hst_data[:,1].max())

### Load ECDFS Data

In [ ]:
obs_butler = Butler(obs_repo, collections='LSSTComCam/runs/DRP/20241101_20241211/w_2024_51/DM-48233')

In [ ]:
# ECDFS Location

# ra = 53.1250000
# dec = -27.8055556
# spherePoint = lsst.geom.SpherePoint(ra*lsst.geom.degrees,
#                                      dec*lsst.geom.degrees)
# t_skymap = test_butler.get('skyMap', skymap='lsst_cells_v1')

# tract = t_skymap.findTract(spherePoint)
# patch = tract.findPatch(spherePoint)
# tract_id = tract.tract_id
# patch_id = patch.getSequentialIndex()
# print(tract_id, patch_id)

In [ ]:
# Need to get all the tracts + patches that span the HST dataset to maximize the counts

t_skymap = obs_butler.get('skyMap', skymap='lsst_cells_v1')
hst_data_locs = []

for ra, dec in zip(hst_data[:,0], hst_data[:,1]):
    spherePoint = lsst.geom.SpherePoint(ra*lsst.geom.degrees,
                                     dec*lsst.geom.degrees)
    tract = t_skymap.findTract(spherePoint)
    patch = tract.findPatch(spherePoint)
    tract_id = tract.tract_id
    patch_id = patch.getSequentialIndex()
    hst_data_locs.append((tract_id, patch_id))
print(tract_id, patch_id)

In [ ]:
# All tracts + patches that we can match against
hst_data_locs = list(set(hst_data_locs))

In [ ]:
pipeline_cat = []

for tract, patch in hst_data_locs:
    pipeline_cat.append(obs_butler.get('objectTable', skymap='lsst_cells_v1', patch=patch, tract=tract,
                           parameters={"columns":['coord_ra', 'coord_dec', 'refExtendedness',
                                                  'detect_isPrimary', 'detect_fromBlend', 'parentObjectId',
                                                  'shape_xx', 'shape_xy', 'shape_yy', 'refBand', 'x', 'y'] + aper_bands + kron_bands}))

In [ ]:
full_cat = pd.concat(pipeline_cat)

In [ ]:
primary_cat = full_cat[full_cat['detect_isPrimary']==True]

In [ ]:
primary_cat = primary_cat.assign(i_aper_mag=u.nJy.to(u.ABmag, primary_cat['i_ap12Flux']))
primary_cat = primary_cat.assign(i_kron_mag=u.nJy.to(u.ABmag, primary_cat['i_kronFlux']))

In [ ]:
# The primary catalog will include points in the patches not in the actual ECDFS field
# Querying for points within 3'' of the HST tree to restrict to the actual ECDFS field

primary_all_coords = np.vstack((primary_cat['coord_ra'], primary_cat['coord_dec'])).T
nearest_dists, _ = hst_tree.query(primary_all_coords)

ecdfs_filt = nearest_dists < 3*arcsec
ecdfs_cat = primary_cat[ecdfs_filt]
ecdfs_coords = primary_all_coords[ecdfs_filt]

In [ ]:
# plt.plot(hst_ra[::10], hst_dec[::10], '.', label="HST")
# plt.plot(ecdfs_cat['coord_ra'][::10], ecdfs_cat['coord_dec'][::10], '.', label="Com Cam")
# plt.legend()

In [ ]:
plt.hist(ecdfs_cat['i_kron_mag'], range=(20, 27), bins=51);
plt.axvline(24.5, ls='--', color='k')
plt.semilogy()
plt.title("ECDFS DRP i-mag Distribution")
plt.xlabel("i-mag")
plt.ylabel("Counts");

Power-law behavior stops at 24.5 which puts the magnitude limit of our sample

In [ ]:
ecdfs_mag_filt = ecdfs_cat['i_kron_mag'] < 24.5
ecdfs_maglimited = ecdfs_cat[ecdfs_mag_filt]
primary_data = ecdfs_coords[ecdfs_mag_filt]
primary_tree = KDTree(primary_data)

In [ ]:
ecdfs_blend, primary_match_array = match_unrec_blends(primary_data, hst_f814_mags, primary_tree, hst_tree, min_mag=33, max_diff=2)

In [ ]:
i_kron_mag = ecdfs_cat['i_kron_mag'].values

In [ ]:
# # What are the brightest pixels between the two sets? They should up match
# hst_brightest = np.argsort(hst_f814_mags)[:500]
# comcam_brightest = np.argsort(i_kron_mag)[:500]

# plt.plot(ecdfs_cat.iloc[comcam_brightest]['coord_ra'], ecdfs_cat.iloc[comcam_brightest]['coord_dec'], '.')
# plt.plot(hst_ra[hst_brightest], hst_dec[hst_brightest], '.', alpha=.5)

In [ ]:
ecdfs_blend, primary_match_array = match_unrec_blends(primary_data, hst_f814_mags, primary_tree, hst_tree, min_mag=30, max_diff=2)

In [ ]:
print(f"We have a total of {len(ecdfs_blend)} objects, {ecdfs_blend.sum()/len(ecdfs_blend) * 100:0.1f}% of which are unrecognized blends using the science pipeline")

In [ ]:
imag_bins = np.arange(18, 25, .15)

In [ ]:
fblend_dict = fblend_generic_1d(ecdfs_maglimited['i_kron_mag'], ecdfs_blend, b_bins=imag_bins)

In [ ]:
# Plot with errorbars
fig,ax = plt.subplots(1, figsize=(6,6))

ytop = .45 # Set this after looking at the plot to make it prettier
ax.errorbar(fblend_dict['bins'],fblend_dict['fraction'], yerr=fblend_dict['err'], marker='.', ls='-')

# ax.legend(frameon=False, bbox_to_anchor=(1.2, .6))
# ax.set_ylim(top=1.0, bottom=.7)
ax.set_ylabel("Fraction of unrec. blends", fontsize=14)
ax.set_xlabel("Object i-mag", fontsize=14);
ax.set_xlim(18.5, 24.5)
ax.set_ylim(-.04, ytop)
# ax.legend(frameon=False, bbox_to_anchor=(1.4,.5))
# ax.legend(loc='upper left', fontsize=14)
ax.set_yticks(np.arange(0,ytop,.05))
ax.axhline(0, ls='--', color='black')
for item in (ax.get_xticklabels() + ax.get_yticklabels()):
    item.set_fontsize(14)
ax.grid(visible=True)
ax.set_title("ECDFS Unrecognized Blends")
# ax.set_title(f"Matching with {min_mag} Magnitude Cutoff and {mag_cutoff} Difference Cutoff")

In [ ]:
# Are we matching to objects that are dimmer to 28mags in HST?
# Only printing the first 5 since there are quite a few....
counter = 0

for i, pma in enumerate(primary_match_array):
    mags = pma[0]
    if (mags>28).any():
        if counter < 5:
            print(i)
        counter += 1
print(f"In total we have {counter} blends with a component fainter than 28")

In [ ]:
# The 12th blend is the 81st object and matches with 2 very faint objects:
primary_match_array[12]

### Extremely Faint HST Match

Let's focus on the 81st object and see what's going on. It might be a spurious DRP detection....

In [ ]:
ra_target = ecdfs_maglimited.iloc[81]['coord_ra']
dec_target = ecdfs_maglimited.iloc[81]['coord_dec']

In [ ]:
spherePoint = lsst.geom.SpherePoint(ra_target*lsst.geom.degrees,
                                    dec_target*lsst.geom.degrees)
tract = t_skymap.findTract(spherePoint)
patch = tract.findPatch(spherePoint)
print(tract.tract_id, patch.getSequentialIndex())

In [ ]:
bands = 'gri'
new_deep_calexps = []
for band in bands:
    new_deep_calexps.append(obs_butler.get('deepCoadd_calexp', skymap='lsst_cells_v1',
                                            band=band, patch=24, tract=5063))#, tract=tract, patch=patch, band=band))

new_wcs = new_deep_calexps[2].getWcs()

coadds = MultibandExposure.fromExposures(bands, new_deep_calexps)

In [ ]:
x_target, y_target = new_wcs.skyToPixelArray(ra_target, dec_target, degrees=True)

In [ ]:
type(coadds)

In [ ]:
showRGB(coadds.image, figsize=(6,6))
plt.plot(x_target, y_target, 'gx')

In [ ]:
# Set up a smaller frame to zoom-in on our actual problem
frame = 250
# sampleBBox = Box2I(Point2I(10700-frame, 9500-frame), Extent2I(63+2*frame, 87+2*frame))
sampleBBox = Box2I(Point2I(x_target[0]-frame, y_target[0]-frame), Extent2I(2*frame,2*frame))

subset = coadds[:, sampleBBox]

In [ ]:
# Showing each band with tiny bounds to increase visibiltiy of low surface brightness objects
fig, ax = plt.subplots(ncols=3, figsize=(18,6))


for i in range(3):
    c = plt.Circle((x_target[0], y_target[0]), radius=12, fill=False)
    ax[i].imshow(subset[bands[i]].image.array[::-1,:], norm='asinh', # Image is just flipped on the vertical axis
               extent=(sampleBBox.getMinX(), sampleBBox.getMaxX(), sampleBBox.getMinY(), sampleBBox.getMaxY()), vmin=-.1, vmax=.1) 
    ax[i].add_patch(c)
    ax[i].set_title(f"{bands[i]}")
    # ax[i].plot(x_target, y_target, 'rx', alpha=0.025)

There is a tiny blip in the $i$-band which might be an object but not super confident. 

Run the `SourceDetectionTask` to see what the task finds

In [ ]:
schema = SourceCatalog.Table.makeMinimalSchema()

In [ ]:
config = SourceDetectionTask.ConfigClass()
# detect sources at 5 sigma 
config.thresholdValue = 5
config.thresholdType = "stdev"

In [ ]:
detectionTask = SourceDetectionTask(schema=schema, config=config)

In [ ]:
table = SourceCatalog.Table.make(schema)
detectionResult = detectionTask.run(table, subset['i'])
catalog = detectionResult.sources

In [ ]:
# showRGB(subset.image)
plt.imshow(subset['i'].image.array[::-1,:], norm='asinh', # Image is just flipped on the vertical axis
           extent=(sampleBBox.getMinX(), sampleBBox.getMaxX(), sampleBBox.getMinY(), sampleBBox.getMaxY()), vmin=-.1, vmax=.1) 
detectNum = 0 

subset_ras = []
subset_decs = []

for i,ca in enumerate(catalog):
    fp = ca.getFootprint()
    for peak in fp.getPeaks():
        plt.plot(peak.getIx(), peak.getIy(), "rx", mew=2, alpha=.15)
        ra, dec = new_wcs.pixelToSky(peak.getFx(), peak.getFy())
        subset_ras.append(ra.asDegrees())
        subset_decs.append(dec.asDegrees())
        # plt.annotate(text='x', xy=(fp.peaks['f_x'], fp.peaks['f_y']), color='white')
        detectNum += 1

assert detectNum == detectionResult.numPosPeaks
plt.title(f"{config.thresholdValue:0.0f}$\sigma$ Detection Threshold $i$-band")

`SourceDetectionTask` does a really good job! How does this compare to what the DRP catalog find?

In [ ]:
subset_ras = np.array(subset_ras)
subset_decs = np.array(subset_decs)

In [ ]:
subset_xs, subset_ys = new_wcs.skyToPixelArray(subset_ras, subset_decs, degrees=True)

In [ ]:
all_x, all_y = new_wcs.skyToPixelArray(ecdfs_maglimited['coord_ra'], ecdfs_maglimited['coord_dec'], degrees=True)

In [ ]:
drp_bbox_filt = sampleBBox.contains(all_x, all_y)
drp_x = all_x[drp_bbox_filt]
drp_y = all_y[drp_bbox_filt]

In [ ]:
# showRGB(subset.image)

plt.title(f"{config.thresholdValue:0.0f}$\sigma$ Detection Threshold $i$-band")

plt.plot(subset_xs, subset_ys, lw=0, marker='o', fillstyle='none', label="SourceDetection", color='red', alpha=.25)
plt.plot(drp_x, drp_y, lw=0, marker='o', fillstyle='none', label="DRP", color='black', alpha=1)
plt.imshow(subset['i'].image.array[::-1,:], norm='asinh', # Image is just flipped on the vertical axis
           extent=(sampleBBox.getMinX(), sampleBBox.getMaxX(), sampleBBox.getMinY(), sampleBBox.getMaxY()), vmin=-.1, vmax=.1) 
plt.legend(frameon=False, bbox_to_anchor=(1.1, .55))

The DRP is mostly missing the very faint objects

What are the objects that are fainter the supposed one at the center? The center object has a `kron_mag` $\approx 24.0$ -- what do the fainter objects look like?

In [ ]:
drp_imags = ecdfs_maglimited['i_kron_mag'][drp_bbox_filt] 

In [ ]:
circle_filt = (drp_imags < ecdfs_maglimited['i_kron_mag'].iloc[81])

In [ ]:
# showRGB(subset.image)
fig, ax = plt.subplots(ncols=3, figsize=(18,6))


for i in range(3):
    c = plt.Circle((x_target[0], y_target[0]), radius=12, fill=False)
    ax[i].imshow(subset[bands[i]].image.array[::-1,:], norm='asinh', # Image is just flipped on the vertical axis
               extent=(sampleBBox.getMinX(), sampleBBox.getMaxX(), sampleBBox.getMinY(), sampleBBox.getMaxY()), vmin=-.1, vmax=.1) 
    ax[i].add_patch(c)
    ax[i].set_title(f"{bands[i]}")
    ax[i].plot(drp_x[circle_filt], drp_y[circle_filt], lw=0, marker='o', alpha=.5, color='r', fillstyle='none', label="DRP")
    # ax[i].plot(x_target, y_target, 'rx', alpha=0.025)

Our object should definitely be visible in the images then

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(15, 5))


for i in range(3):
    c = plt.Circle((x_target[0], y_target[0]), radius=12, fill=False, color='g')

    masked = subset[bands[i]].getMaskedImage().mask
    ax[i].imshow(masked.array[::-1,:] & pow(2,5),
                extent=(sampleBBox.getMinX(), sampleBBox.getMaxX(), sampleBBox.getMinY(), sampleBBox.getMaxY()))
    ax[i].set_title(f"Detected pixels in {bands[i]}")
    ax[i].add_patch(c)

#### DynamicDetectionTask

In [ ]:
schema = SourceCatalog.Table.makeMinimalSchema()

raerr = schema.addField("coord_raErr", type="F")
decerr = schema.addField("coord_decErr", type="F")

In [ ]:
config = DynamicDetectionTask.ConfigClass()
dynamicTask = DynamicDetectionTask(schema=schema, config=config)

In [ ]:
config = SingleFrameMeasurementTask.ConfigClass()
sourceMeasurementTask = SingleFrameMeasurementTask(schema=schema, config=config)

In [ ]:
table = SourceCatalog.Table.make(schema)

In [ ]:
i_subset = new_deep_calexps[2].subset(sampleBBox)

In [ ]:
dynamicResult = dynamicTask.run(table, i_subset) # clearMask=True by default so we don't need to remove the mask.

In [ ]:
catalog = dynamicResult.sources

In [ ]:
sourceResult = sourceMeasurementTask.run(measCat=catalog, exposure=i_subset)

In [ ]:
# catalog = catalog.copy()

In [ ]:
print(len(catalog))

In [ ]:
# showRGB(subset.image)
plt.imshow(subset['i'].image.array[::-1,:], norm='asinh', # Image is just flipped on the vertical axis
           extent=(sampleBBox.getMinX(), sampleBBox.getMaxX(), sampleBBox.getMinY(), sampleBBox.getMaxY()), vmin=-.1, vmax=.1) 
detectNum = 0 

dynamic_subset_ras = []
dynamic_subset_decs = []

for i,ca in enumerate(catalog):
    fp = ca.getFootprint()
    for peak in fp.getPeaks():
        plt.plot(peak.getIx(), peak.getIy(), "rx", mew=2, alpha=.5)
        ra, dec = new_wcs.pixelToSky(peak.getFx(), peak.getFy())
        dynamic_subset_ras.append(ra.asDegrees())
        dynamic_subset_decs.append(dec.asDegrees())
        # plt.annotate(text='x', xy=(fp.peaks['f_x'], fp.peaks['f_y']), color='white')
        detectNum += 1
print(f"We detect {detectNum} objects")

In [ ]:
tdists = np.sqrt((catalog['base_SdssCentroid_x'] - x_target[0])**2 + (catalog['base_SdssCentroid_y'] - y_target[0])**2)

In [ ]:
target_ndx = tdists.argmin()
ecdfs_ndx = 81
print(target_ndx, tdists[target_ndx])

In [ ]:
print(f"12pix aperture flux of target from DRP: {ecdfs_maglimited.iloc[ecdfs_ndx]['i_ap12Flux']:0.02f} vs SourceMeasurement: {catalog[target_ndx]['base_CircularApertureFlux_12_0_instFlux']:0.02f}")

Let's see if we are getting the "correct" (or at least closer) flux on a brighter object

In [ ]:
plt.imshow(subset['i'].image.array[::-1,:], norm='asinh', # Image is just flipped on the vertical axis
           extent=(sampleBBox.getMinX(), sampleBBox.getMaxX(), sampleBBox.getMinY(), sampleBBox.getMaxY()), vmin=-.1, vmax=.1) 

plt.plot(12210, 8013, 'rx')

In [ ]:
tdists = np.sqrt((catalog['base_SdssCentroid_x'] - 12210)**2 + (catalog['base_SdssCentroid_y'] - 8013)**2)
e_dists = np.sqrt((drp_x - 12210)**2 + (drp_y - 8013)**2)

In [ ]:
target_ndx = tdists.argmin()
ecdfs_ndx = e_dists.argmin()
print(target_ndx, tdists[target_ndx])

In [ ]:
plt.imshow(subset['i'].image.array[::-1,:], norm='asinh', # Image is just flipped on the vertical axis
           extent=(sampleBBox.getMinX(), sampleBBox.getMaxX(), sampleBBox.getMinY(), sampleBBox.getMaxY()), vmin=-.1, vmax=.1) 

plt.plot(catalog[target_ndx]['base_SdssCentroid_x'], catalog[target_ndx]['base_SdssCentroid_y'], 'rx')
plt.plot(drp_x[ecdfs_ndx], drp_y[ecdfs_ndx], marker='x', color='purple')

In [ ]:
catalog[target_ndx]

In [ ]:
print(f"12pix aperture flux of target from DRP: {ecdfs_maglimited.iloc[ecdfs_ndx]['i_ap12Flux']:0.02f} vs SourceMeasurement: {catalog[target_ndx]['base_CircularApertureFlux_12_0_instFlux']:0.02f}")

### Save data for Source Extracting

In [ ]:
plt.imshow(img.image.array[::-1,:], norm='asinh', vmin=-1, vmax=1)

In [ ]:
bands = 'gri'
big_deep_calexps = []
for tract, patch in hst_data_locs:
    primary_hdu = fits.PrimaryHDU()
    hdu_list = [primary_hdu]
    
    for band in bands:
        img = obs_butler.get('deepCoadd_calexp', skymap='lsst_cells_v1', band=band, patch=patch, tract=tract)#, tract=tract, patch=patch, band=band))
        image_hdu = fits.ImageHDU(data=img.image.array[::-1,:], name=band)
        hdu_list.append(image_hdu)
    hdul = fits.HDUList(hdu_list)
    hdul.writeto(f'/home/a/adari/DATA/ECDFS_ComCam_{tract}_{patch}.fits')

In [ ]:
ecdfs_cat.to_pickle('/home/a/adari/DATA/ecdfs_catalog.pkl')

In [ ]:
ccdvisit = obs_butler.get('ccdVisitTable', instrument='LSSTComCam')

In [ ]:
kale = img.getInfo()

In [ ]:
snake = kale.getPhotoCalib()

In [ ]:
-2.5*np.log10(snake.getInstFluxAtZeroMagnitude())

In [ ]:
g_img = obs_butler.get('deepCoadd_calexp', skymap='lsst_cells_v1', band='i', patch=34, tract=5063)#, tract=tract, patch=patch, band=band))

In [ ]:
-2.5*np.log10(g_img.getInfo().getPhotoCalib().getInstFluxAtZeroMagnitude())

In [ ]:
hst_data_locs[-1]

In [ ]:
i_begins = {}

for tract, patch in hst_data_locs:
    i_img = obs_butler.get('deepCoadd_calexp', skymap='lsst_cells_v1', band='i', patch=patch, tract=tract)
    

In [ ]:
i_img = obs_butler.get('deepCoadd_calexp', skymap='lsst_cells_v1', band='i', patch=34, tract=5063)#, tract=tract, patch=patch, band=band))

In [ ]:
print(i_img.getBBox().beginX, i_img.getBBox().beginY)

In [ ]:
kale.beginX

In [ ]:
kale.beginY

In [ ]:
ecdfs_cat